# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine, inspect

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score

import pickle

In [4]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
inspector = inspect(engine)
print(inspector.get_table_names(),'\n')

df = pd.read_sql_table('Messages', engine)
print(df.head())
print('Columns in the dataframe:')
for col in df.columns:
    print(f'{col}')

['Messages'] 

   id                                            message   genre  related  \
0   2  Weather update - a cold front from Cuba that c...  direct        1   
1   7            Is the Hurricane over or is it not over  direct        1   
2   8                    Looking for someone but no name  direct        1   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  12  says: west side of Haiti, rest of the country ...  direct        1   

   request  offer  aid_related  medical_help  medical_products  \
0        0      0            0             0                 0   
1        0      0            1             0                 0   
2        0      0            0             0                 0   
3        1      0            1             0                 1   
4        0      0            0             0                 0   

   search_and_rescue  ...  aid_centers  other_infrastructure  weather_related  \
0                  0  ...            0      

In [5]:
# define feature variable X (the message text)
X = df['message']

# define target variable Y (the categories) and drop columns that are not target variables: 'id', 'message', 'genre'
Y = df.drop(columns=['id', 'message', 'genre'])

print(X.shape)
print(Y.shape)

(26216,)
(26216, 36)


### 2. Write a tokenization function to process your text data

In [6]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def tokenize(text):
    """
    Tokenization function to process text data.

    Inputs:
    text(str): The input text to be tokenized and processed.

    Returns:
    list: A list of clean, processed tokens (words) from the input text.
    """
    # normalize text: convert to lowercase and remove punctuation
    text = text.lower() # convert text to lowercase to standardize it
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) # replace non-alphanumeric characters with spaces

    # tokenize the text into words
    tokens = word_tokenize(text) # split the text into individual words (tokens)

    # remove stopwords (common words that don't add much meaning)
    tokens = [word for word in tokens if word not in stopwords.words('english')]

    # lemmatize: convert words to their base form
    lemmatizer = WordNetLemmatizer() # initialize the lemmatizer
    clean_tokens = [lemmatizer.lemmatize(token) for token in tokens] # lematize each token

    return clean_tokens

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leylaalieva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leylaalieva/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leylaalieva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
# build the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
# split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# train the pipeline
pipeline.fit(X_train, Y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x12ec20540>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
# predict on the test set
Y_pred = pipeline.predict(X_test)

# print classification report for each target category
for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    print(classification_report(Y_test[col], Y_pred[:,i]))

Category: related
              precision    recall  f1-score   support

           0       0.71      0.42      0.53      1266
           1       0.83      0.94      0.88      3938
           2       0.38      0.38      0.38        40

    accuracy                           0.81      5244
   macro avg       0.64      0.58      0.60      5244
weighted avg       0.80      0.81      0.79      5244

Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.49      0.62       895

    accuracy                           0.90      5244
   macro avg       0.87      0.73      0.78      5244
weighted avg       0.89      0.90      0.88      5244

Category: offer


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

Category: aid_related
              precision    recall  f1-score   support

           0       0.80      0.85      0.82      3113
           1       0.75      0.68      0.72      2131

    accuracy                           0.78      5244
   macro avg       0.77      0.77      0.77      5244
weighted avg       0.78      0.78      0.78      5244

Category: medical_help
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4822
           1       0.65      0.09      0.16       422

    accuracy                           0.92      5244
   macro avg       0.79      0.54      0.56      5244
weighted avg       0.90      

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-pa

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5021
           1       0.00      0.00      0.00       223

    accuracy                           0.96      5244
   macro avg       0.48      0.50      0.49      5244
weighted avg       0.92      0.96      0.94      5244

Category: weather_related
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      3806
           1       0.85      0.69      0.76      1438

    accuracy                           0.88      5244
   macro avg       0.87      0.82      0.84      5244
weighted avg       0.88      0.88      0.88      5244

Category: floods
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4833
           1       0.90      0.46      0.61       411

    accuracy                           0.95      5244
   macro avg       0.93      0.73      0.79      5244
weighted avg       0.95      0.

The classification report above shows that in many categories (e.g. offer, shops, tools) the number of messages labeled as 1 is extremeky small  compared to those labeled as 0. This leads to poor performance in those categories as the model has difficulty learning from such imbalanced data.<br>
Categories like related, request and aid_related, which have more balanced distributions of 0s and 1s, show much better precision, recall and F1-scores.<br>

I'd therefore address the class imbalance first. I will explore the techniques like oversampling the minority class or undersampling the majority class to balance the data.

Unfortunately I haven't found a robust working solution for making the dataset more balanced. Due to time constraints, I focused on building a clean and functional pipeline. However, the dataset exhibited class and label imbalance that likely impacted the model's performance, especially in minority categories.<br>
Given more time, I would explore techniques like oversampling, undersampling or SMOT to balance the dataset and make the model's prediction more reliable for real-world scenarios. 

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# define the parameters for GridSearch
parameters = {
    'clf__estimator__n_estimators': [100],
    'clf__estimator__min_samples_split': [2,4]
}

# create GridSearch object
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=1)

# fit the model
cv.fit(X_train, Y_train)

# predict on the test set using the optimized model
Y_pred = cv.predict(X_test)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.263 total time=12.1min


/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.265 total time=12.2min


...

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 4/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=100;, score=0.253 total time=10.9min


/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 5/5] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=100;, score=0.274 total time=10.3min


/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# evaluate the optimized model using classification report
for i, col in enumerate(Y.columns):
    print(f'Category: {col}')
    print(classification_report(Y_test[col], Y_pred[:,i]))

    # show the best parameters
    print('Best parameters found by GridSearchCV:', cv.best_params_)

Category: related
              precision    recall  f1-score   support

           0       0.70      0.43      0.53      1266
           1       0.83      0.94      0.88      3938
           2       0.39      0.40      0.40        40

    accuracy                           0.81      5244
   macro avg       0.64      0.59      0.60      5244
weighted avg       0.80      0.81      0.79      5244

Best parameters found by GridSearchCV: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}
Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.49      0.62       895

    accuracy                           0.90      5244
   macro avg       0.87      0.74      0.78      5244
weighted avg       0.89      0.90      0.89      5244

Best parameters found by GridSearchCV: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}
Category: offer
      

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5089
           1       0.73      0.07      0.13       155

    accuracy                           0.97      5244
   macro avg       0.85      0.54      0.56      5244
weighted avg       0.97      0.97      0.96      5244

Best parameters found by GridSearchCV: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}
Category: child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5244

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

Best parameters found by GridSearchCV: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}
Category: water
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4905
           1       

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-pa

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5198
           1       0.00      0.00      0.00        46

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.98      0.99      0.99      5244

Best parameters found by GridSearchCV: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}
Category: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.00      0.00      0.00        22

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

Best parameters found by GridSearchCV: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}
Category: aid_centers
              precision    recall  f1-score   support

           0       

After tuning the model showed minimal changes in precision and recall across categies suggesting that hyperparameter adjustments had limited impact on the performance.<br>
Rare categories continue to have low recall, highlighting that class imbalance remains a key challenge beyond parameter tuning.

In [25]:
print(overall_accuracy)

0.9485072887532842


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
extended_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer()),
            ('svd', TruncatedSVD(n_components=100))
        ]))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, min_samples_split=2)))
])

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

extended_pipeline.fit(X_train, Y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(ngram_range=(1,
                                                                                               2),
                                                                                  tokenizer=<function tokenize at 0x12ec20540>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer()),
                                                                 ('svd',
                                                                  TruncatedSVD(n_components=100))]))])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [30]:
Y_pred_extended = extended_pipeline.predict(X_test)

for i, col in enumerate(Y_test.columns):
    print(f'Category: {col}')
    print(classification_report(Y_test[col], Y_pred_extended[:,i]))

extended_accuracy = (Y_pred_extended == Y_test.values).mean()
print(extended_accuracy)

Category: related
              precision    recall  f1-score   support

           0       0.69      0.32      0.43      1266
           1       0.81      0.96      0.88      3938
           2       0.80      0.10      0.18        40

    accuracy                           0.79      5244
   macro avg       0.76      0.46      0.50      5244
weighted avg       0.78      0.79      0.76      5244

Category: request
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      4349
           1       0.75      0.56      0.64       895

    accuracy                           0.89      5244
   macro avg       0.83      0.76      0.79      5244
weighted avg       0.89      0.89      0.89      5244

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.5

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.97      0.98      0.98      4905
           1       0.71      0.58      0.64       339

    accuracy                           0.96      5244
   macro avg       0.84      0.78      0.81      5244
weighted avg       0.95      0.96      0.96      5244

Category: food
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      4649
           1       0.79      0.62      0.69       595

    accuracy                           0.94      5244
   macro avg       0.87      0.80      0.83      5244
weighted avg       0.93      0.94      0.93      5244

Category: shelter
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      4774
           1       0.78      0.40      0.53       470

    accuracy                           0.94      5244
   macro avg       0.86      0.70      0.75      5244
weighted avg       0.93      0.94      0.

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-pa

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      5073
           1       0.00      0.00      0.00       171

    accuracy                           0.97      5244
   macro avg       0.48      0.50      0.49      5244
weighted avg       0.94      0.97      0.95      5244

Category: death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5007
           1       0.71      0.19      0.30       237

    accuracy                           0.96      5244
   macro avg       0.84      0.59      0.64      5244
weighted avg       0.95      0.96      0.95      5244

Category: other_aid
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      4549
           1       0.57      0.09      0.15       695

    accuracy                           0.87      5244
   macro avg       0.72      0.54      0.54      5244
weighted avg       0.84      0.87     

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-pa

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5177
           1       0.00      0.00      0.00        67

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99      0.98      5244

Category: other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5021
           1       0.50      0.00      0.01       223

    accuracy                           0.96      5244
   macro avg       0.73      0.50      0.49      5244
weighted avg       0.94      0.96      0.94      5244

Category: weather_related
              precision    recall  f1-score   support

           0       0.86      0.93      0.90      3806
           1       0.77      0.62      0.68      1438

    accuracy                           0.84      5244
   macro avg       0.82      0.77      0.79      5244
weighted avg     

The extended model achieves the overall accurancy of 94,47%, similar to the previously tuned model. Adding bigrams and dimensionality reduction has yielded slight improvements for some categories but still has difficulty with rare classes due to class imbalance.

### 9. Export your model as a pickle file

In [35]:
with open('../models/classifier.pkl', 'wb') as file:
    pickle.dump(extended_pipeline, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.